```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Passive drift functionality on GPU

This notebook serves as the laboratory for prototyping implementations of passive drift of an object along simulation results from our simplified ocean models.

These drift functions will be used to speed up our basic particle filter application, trying to predict passive drift in a chaotic ocean - see BasicParticleFilter.ipynb.

The goals will be
- [ ] Make a singleGPUPassiveDrifterEnsamble class, holding data positions on the GPU, but where data assimilation part is executed on the CPU.
- [ ] ???

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import GPUDrifterCollection
from SWESimulators import DrifterEnsemble
from SWESimulators import DataAssimilationUtils as dautils

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

## Thoughts on code structure

The observation will in these initial cases be a chosen model realization. When initializing a data assimilation with N particles, N+1 particles should be created and distributed on simulators.

One hypothesis for our ocean simulator is that integrating 100 particles within the same simulation is equally expensive as integrating 1 particle. Each particle integration should be done with a single thread on the GPU, so all 100 particles will can be processed in parallel.

If this assumption is true, it is best to have all particle positions continuous in memory. Hence, it will be implemented as an struct of array.

## Create random particles, and create random observation
**(This text is mainly copy-paste from BasicParticleFilter.ipynb)**

Particles are created in a GlobalParticle class, which holds the positions of all ensemble member particles, and one additional particle which serves as our observation.
This class should have all computational functionality that relies on the relationships between particles and the observation, and the ensemble itself.

Filtering and resampling of particles does not belong in this class.

List of functions that could be useful (and which of them are implemented):
- [ ] initialize uniform on unit square
- [ ] 
- [w] initialize gaussian 
- [w] Calculate distances from observation
- [w] get weights from Gaussian distribution
- [w] get weights from Cauchy distribution
- [w] find ensemble mean position
- [w] find ensemble variance
- [w] set observation to a given coordinate
- [w] copy function

**About distances**: In order to calculate distances in a unified way, information about boundary conditions needs to be known by the class. E.g., the distance between a particle at $(0.99, 0.99)$ from an observation at $(0.01, 0.02)$ on a unit square domain, is about $\sqrt{2}$. However, with periodic boundary conditions, their distance is only $0.05$.

Let the domain size be $(L_x, L_y)$, and define the particle and observation positions as $(x_p, y_p)$ and $(x_o, y_o)$, respectively.
The minimal distance with a periodic boundary can then be found by
$$ d_{x,min} = \min \left\{ |x_p - x_o|, |(x_p - L_x) - x_o|, |(x_p + L_x) - x_o| \right\}$$
$$ d_{y,min} = \min \left\{ |y_p - y_o|, |(y_p - L_y) - y_o|, |(y_p + L_y) - y_o| \right\}$$
and 
$$ d_{min} = \sqrt{ d_{x,min}^2 + d_{y,min}^2}$$

** About ensemble mean position**: When finding the mean position of particles, the above considerations needs to be taken as well. The ensemble mean position should be found by the coordinate position which results in the minimal distance given above.
In other words, the position we should consider for the mean is
$$ x^*_p = {\arg\min}_{x \in \{x_p, x_p \pm L_x \}} |x - x_o|, $$
$$ y^*_p = {\arg\min}_{y \in \{y_p, y_p \pm L_y \}} |y - y_o|. $$

## The SingleGPUPassiveDrifterEnsemble class

In [ ]:
poses = particlesHost = np.zeros((10 + 1, 2))
print poses.shape
print poses.dtype
poses = poses.astype(np.float32, order='C')
print poses.dtype

In [ ]:
# Read some simulator field from netcfd:
netcdf_file_name = "netcdf_2018_02_12/CDKLM16_2018_02_12-16_52_49.out"
reload(CDKLM16)
if 'sim' in globals():
    sim.cleanUp()
sim = CDKLM16.CDKLM16.fromfilename(cl_ctx, \
                                   netcdf_file_name, \
                                   cont_write_netcdf=False)
#eta0, hu0, hv0 = sim.download()
Hi = sim.downloadBathymetry()[0]

In [ ]:
reload(DrifterEnsemble)
#Clean up old simulator if any:
if 'gpuParticles' in globals():
    gpuParticles.cleanUp()
    
numParticles = 100
observation_variance = 5*sim.dx
gpuParticles = GPUDrifterCollection.GPUDrifterCollection(cl_ctx, numParticles,
                                                         observation_variance=observation_variance,
                                                         boundaryConditions=sim.boundary_conditions,
                                                         domain_size_x=sim.nx*sim.dx,
                                                         domain_size_y=sim.ny*sim.dy)
gpuParticles.initializeUniform()
fig = plt.figure(figsize=(5,5))
# Plot host data:
plt.plot(gpuParticles.driftersHost[:-1,0], \
         gpuParticles.driftersHost[:-1,1], 'yo')
plt.plot(gpuParticles.driftersHost[-1,0], \
         gpuParticles.driftersHost[-1,1], 'ro')
# Plot device data:
plt.plot(gpuParticles.getDrifterPositions()[:,0], \
         gpuParticles.getDrifterPositions()[:,1], 'k+')
plt.plot(gpuParticles.getObservationPosition()[0], \
         gpuParticles.getObservationPosition()[1], 'k+')

# Drift
for i in range(20):
    for j in range(1000):
        gpuParticles.drift(sim.cl_data.h0, sim.cl_data.hu0, sim.cl_data.hv0, np.max(Hi), \
                           sim.nx, sim.ny, sim.dx, sim.dy, sim.dt, \
                           np.int32(2), np.int32(2))
    
    # Plot device data:
    plt.plot(gpuParticles.getDrifterPositions()[:,0], \
             gpuParticles.getDrifterPositions()[:,1], 'r.')
    plt.plot(gpuParticles.getObservationPosition()[0], \
             gpuParticles.getObservationPosition()[1], 'b.')

print gpuParticles.getDomainSizeX()
gpuParticles.plotDistanceInfo()

In [ ]:
numParticles = 20
observation_variance = 0.1
gpu2 = GPUDrifterCollection.GPUDrifterCollection(cl_ctx, numParticles,
                                                 observation_variance=observation_variance,
                                                 boundaryConditions=sim.boundary_conditions)
gpu2.initializeUniform()
gpu2.plotDistanceInfo()

partpos = gpu2.getDrifterPositions()
partpos = partpos*0.3 + 0.5 - 0.3/2
gpu2.setDrifterPositions(partpos)
gpu2.setObservationPosition(np.array([0.5, 0.9]))
gpu2.plotDistanceInfo()



## Test resampling

In [ ]:
reload(DrifterEnsemble)
numParticles = 100
observation_variance = 5*sim.dx
dummyEnsemble = DrifterEnsemble.DrifterEnsemble(cl_ctx, numParticles,
                                                observation_variance=observation_variance)

dummyEnsemble.setGridInfo(sim.nx, sim.ny, sim.dx, sim.dy, sim.dt, sim.boundary_conditions)

dummyEnsemble.init()
dummyEnsemble.plotDistanceInfo("Prior resampling")

dautils.probabilisticResampling(dummyEnsemble, reinitialization_variance=10*sim.dx)
dummyEnsemble.enforceBoundaryConditions()
dummyEnsemble.plotDistanceInfo("After resampling")



# Passive drift trajectories in the SWE simulators

Here, we will make a naive implementation of particles drifting within our simplified ocean models.
For simplicity, a non-staggered implementation is chosen, as it makes it easier to evaluate the velocity field.


In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 50
ny = 50

dx = 4.0
dy = 4.0

dt = 0.1
g = 9.81

f = 0.5

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Add disturbance:
addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05, validDomain)
addBump(eta0, nx, ny, dx, dy, 0.7, 0.2, 0.10, validDomain)
addBump(eta0, nx, ny, dx, dy, 0.1, 0.8, 0.03, validDomain)
eta0 = eta0*0.3

#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
#x_coords = np.subtract(x_coords, x_center)
#y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))



In [ ]:
# Init particles
reload(DrifterEnsemble)
reload(PlotHelper)
np.random.seed(1)
numParticles = 50
observation_variance = 5*dx
resample_variance = 5*dx

## Define a bunch of particles to be released within the given domain
drifterEnsemble = DrifterEnsemble.DrifterEnsemble(cl_ctx, numParticles,
                                             observation_variance=observation_variance)
drifterEnsemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions, eta=eta0)
drifterEnsemble.setParameters(f=f)
drifterEnsemble.init()
drifterEnsemble.plotDistanceInfo(title="Initial particles")

eta1, hu1, hv1 = drifterEnsemble.sim.download()
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

plotter.showDrifters(drifterEnsemble.sim.drifters)

T = 200
sub_t = 1000*dt
#loopsPerFrame = 10
drifterCollections = [drifterEnsemble.sim.drifters.copy()]
plotTitles = ["Initil ensemble"]

def animate(i):
    if (i>0):
        t = drifterEnsemble.step(sub_t)
    else:
        t = 0.0

    eta1, hu1, hv1 = drifterEnsemble.sim.download()
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    
    plotter.showDrifters(drifterEnsemble.sim.drifters)
     
    fig.suptitle("CDKLM16 with GPU drift and residualResampling - Time = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    if (i%50 == 0 and i > 0):
        drifterCollections.append(drifterEnsemble.sim.drifters.copy())
        plotTitles.append("Before particle filter at t = " + str(t))
        
        dautils.residualSampling(drifterEnsemble, reinitialization_variance=resample_variance)
         
        drifterCollections.append(drifterEnsemble.sim.drifters.copy())
        plotTitles.append("After particle filter at t = " + str(t))
    
    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax hu: " + str(np.max(hu1)) + \
        "\tMax hv: " + str(np.max(hv1))
        print "\t\tObservation pos: ", drifterEnsemble.observeTrueState()
                     
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

### Create a field to start from:
#sim.step(dt*T*10)
#eta1, hu1, hv1 = sim.download()
#plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
#             hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
#             hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
#sim.cleanUp()

In [ ]:
for i in range(len(drifterCollections)):
    drifterCollections[i].plotDistanceInfo(title=plotTitles[i])